In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.distributions import Dirichlet, Bernoulli, Uniform
import pandas as pd
import tqdm as tm

from src import Simulation as sim
from src import Dir_Reg
from src import Align
from src import visualize_latent_space as vls

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Below we generate the synthetic data set that shows how the latent position in ABCDPRGM evolves through time under different settings.

In [44]:
model = sim.ABC(time = 30,
                nodes = 1200,
                beta = [1,1,-4,5],
                alpha_0 = [[1, 1, 10], [1, 10, 1], [10, 1, 1]])

Let $\widehat{B} \in \mathbb{R}^{q \times p}$ be the MLE that corresponds to the design matrix $X \otimes I_p$, and $\tilde{\beta} = (C^T C)^{-1} C^T \widehat{B}$. Let $\widehat{\beta}$ be the MLE that corresponds to the design matrix $(X \otimes I_p)C$.

We first do Monte Carlo simulations to verify the asymptotic behavior of $\widehat{B}$ and $\tilde{\beta}$. 

In [4]:
torch.manual_seed(0)
model = sim.ABC(time = 2,
                nodes = 1200,
                beta = [1,1,-4,5],
                alpha_0 = [[1, 1, 10], [1, 10, 1], [10, 1, 1]])

In [36]:
def fix_nodes_n_iter(model, nodes, n_iter, constrained):

    """ run the model with n nodes n_iter number of times, record the estimated beta and fisher's information """

    model.update_settings(nodes = nodes)
    
    B = model.settings.B
    beta = model.settings.beta

    q, p = B.shape
    constraint = Dir_Reg.fit.gen_constraint(p, True)

    B_hat = torch.zeros(n_iter, q*p)

    if constrained: 
        qp = 4
    else:
        qp = q*p

    fish_est = torch.zeros(n_iter, (qp)**2)

    for i in range(n_iter):

        torch.manual_seed(i)
        model.update_settings()
        
        Z0 = model.synth_data["lat_pos"][0,]
        Z1 = model.synth_data["lat_pos"][1,]
        Y0 = model.synth_data["obs_adj"][0,]
        X0 = sim.ABC.gen_X(Y0, Z0, model.settings.K)

        est = Dir_Reg.fit(predictor = X0, response = Z1, constrained = constrained, beta_guess = model.settings.beta)

        B_hat[i,] = est.est_result["estimate"].reshape(-1)
        fish_est[i,] = est.est_result["fisher_info"].reshape(-1)

    def to_pd_df(n, mat, b_real, name):
        """ so that the experiment result can live in the DLS data wonderland """
        n_iter, qp = mat.shape
        vec = mat.reshape(1, -1)
        comp = torch.arange(1, qp+1).repeat(n_iter).unsqueeze(dim = 0)
        seed_id = torch.arange(n_iter).repeat_interleave(qp).unsqueeze(dim = 0)
        node_id = n * torch.ones(qp*n_iter).unsqueeze(dim = 0)
        b_real_stack = torch.stack([b_real.reshape(-1)]* n_iter).reshape(1, -1)
        b_df = torch.cat([seed_id, node_id, comp, vec, b_real_stack], dim = 0).T
        b_df = pd.DataFrame(b_df)
        b_df.columns = ["Seed", "Nodes", "Comp", name + "_hat", name + "_real"]
        for column in b_df.columns:
            if "_" not in column:
                try:
                    b_df[column] = b_df[column].astype(int)
                except ValueError:
                    pass

        return(b_df)

    B_df = to_pd_df(nodes, B_hat, B, "B")
    beta_tilde = torch.linalg.solve(constraint.T @ constraint, constraint.T) @ B_hat.T
    beta_df = to_pd_df(nodes, beta_tilde.T, beta, "beta")

    result_dict = {"B_hat": B_df, "beta_tilde": beta_df, "fish_est": fish_est.mean(dim = 0).reshape(qp, qp)}
    return(result_dict)


In [43]:
model = sim.ABC(time = 2,
                nodes = 1200,
                beta = [1,1,-4,5],
                alpha_0 = [[1, 1, 10], [1, 10, 1], [10, 1, 1]])
n_iter = 1
n_set = list(range(3000, 13500, 1500))
constraint = Dir_Reg.fit.gen_constraint(3, True)
constrained = False

temp = fix_nodes_n_iter(model, 1500, n_iter, constrained)
df_B = temp["B_hat"]
df_beta = temp["beta_tilde"]
df_fish = temp["fish_est"].unsqueeze(dim = 0)

for n in n_set:
    temp = fix_nodes_n_iter(model, n, n_iter, constrained)
    df_B = pd.concat([df_B, temp["B_hat"]], ignore_index= True)
    df_beta = pd.concat([df_beta, temp["beta_tilde"]], ignore_index= True)
    df_fish = torch.cat([df_fish, temp["fish_est"].unsqueeze(dim = 0)], dim = 0)

df_vec_fish = pd.DataFrame(df_fish.reshape(8, -1))
df_fish_diag = torch.stack([df_fish[i].diag() for i in range(len(n_set) + 1)])
df_fish_diag = pd.DataFrame(df_fish_diag)

df_B.to_csv(r"simulated_data\emp_var_vs_obs_var\B_oracle.csv", index = False)
df_beta.to_csv(r"simulated_data\emp_var_vs_obs_var\B_oracle_proj.csv", index = False)
df_vec_fish.to_csv(r"simulated_data\emp_var_vs_obs_var\B_fish.csv", index = False)